# Data Loading

In [0]:
!pip install  kagglehub

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import os
dbutils.fs.mkdirs("/Workspace/Users/kedarvaidya0504@gmail.com/kaggle")
os.environ["KAGGLEHUB_CACHE_DIR"] = "/Workspace/Users/kedarvaidya0504@gmail.com/kaggle"

import kagglehub

# Download latest version
path = kagglehub.dataset_download("khandelwalmanas/daily-commodity-prices-india")

print("Path to dataset files:", path)


In [0]:
!cp -r /home/spark-32071c93-e691-42b5-97a4-9c/.cache/kagglehub/datasets/khandelwalmanas/daily-commodity-prices-india/versions/22 /Workspace/Users/kedarvaidya0504@gmail.com

cp: error writing '/Workspace/Users/kedarvaidya0504@gmail.com/22/csv/2024.csv': File too large
cp: failed to close '/Workspace/Users/kedarvaidya0504@gmail.com/22/csv/2024.csv': File too large
cp: error writing '/Workspace/Users/kedarvaidya0504@gmail.com/22/csv/2025.csv': File too large
cp: failed to close '/Workspace/Users/kedarvaidya0504@gmail.com/22/csv/2025.csv': File too large


In [0]:
%sql
CREATE VOLUME workspace.default.pyspark;

In [0]:
dbutils.fs.cp(
    "/Workspace/Users/kedarvaidya0504@gmail.com/22/parquet/",
    "dbfs:/Volumes/workspace/default/pyspark/",
    recurse=True
)


True

In [0]:
import pyspark.sql.functions as F

bad_files = []

for p in paths:
    try:
        df_test = spark.read.parquet(p)
        print("OK:", p)
    except Exception as e:
        print("BAD:", p)
        bad_files.append(p)


OK: dbfs:/Volumes/workspace/default/pyspark/_SUCCESS
OK: dbfs:/Volumes/workspace/default/pyspark/_committed_5458989634332253454
OK: dbfs:/Volumes/workspace/default/pyspark/_committed_7275890549864832928
OK: dbfs:/Volumes/workspace/default/pyspark/_started_5458989634332253454
OK: dbfs:/Volumes/workspace/default/pyspark/_started_7275890549864832928
OK: dbfs:/Volumes/workspace/default/pyspark/part-00000-tid-7275890549864832928-e5dd427b-4466-49fb-8a41-bd4a6b6f9b3a-645-1.c000.snappy.parquet
OK: dbfs:/Volumes/workspace/default/pyspark/part-00001-tid-7275890549864832928-e5dd427b-4466-49fb-8a41-bd4a6b6f9b3a-649-1.c000.snappy.parquet
OK: dbfs:/Volumes/workspace/default/pyspark/part-00002-tid-7275890549864832928-e5dd427b-4466-49fb-8a41-bd4a6b6f9b3a-650-1.c000.snappy.parquet
OK: dbfs:/Volumes/workspace/default/pyspark/part-00003-tid-7275890549864832928-e5dd427b-4466-49fb-8a41-bd4a6b6f9b3a-651-1.c000.snappy.parquet
OK: dbfs:/Volumes/workspace/default/pyspark/part-00004-tid-7275890549864832928-e5dd

In [0]:
import pyspark.sql.functions as F

paths = [
    f.path for f in dbutils.fs.ls("dbfs:/Volumes/workspace/default/pyspark")
    if f.path.endswith(".parquet")
]

dfs = []
for p in paths:
    df_i = (
        spark.read.parquet(p)
        .withColumn("Min_Price", F.col("Min_Price").cast("double"))
        .withColumn("Modal_Price", F.col("Modal_Price").cast("double"))
        .withColumn("Max_Price", F.col("Max_Price").cast("double"))
    )
    dfs.append(df_i)

df_clean = dfs[0]
for d in dfs[1:]:
    df_clean = df_clean.unionByName(d)

display(df_clean.head(5))


State,District,Market,Commodity,Variety,Grade,Arrival_Date,Min_Price,Max_Price,Modal_Price,Commodity_Code
Karnataka,Dharwad,Hubli (Amaragol),Bengal Gram (Gram)(Whole),Average (Whole),FAQ,2001-01-10,1400.0,1400.0,1400.0,6
Karnataka,Dharwad,Hubli (Amaragol),Black Gram (Urd Beans)(Whole),Black Gram (Whole),FAQ,2001-01-10,1380.0,1380.0,1380.0,8
Karnataka,Dharwad,Hubli (Amaragol),Green Gram (Moong)(Whole),Green (Whole),FAQ,2001-01-10,1600.0,1900.0,1800.0,9
Karnataka,Dharwad,Hubli (Amaragol),Groundnut,Gejje,FAQ,2001-01-10,601.0,1181.0,1105.0,10
Karnataka,Dharwad,Hubli (Amaragol),Maize,Local,FAQ,2001-01-10,420.0,445.0,440.0,4


In [0]:
%sql
CREATE VOLUME workspace.default.pyspark_clean;

In [0]:
df_clean.write.mode("overwrite").parquet("/Volumes/workspace/default/pyspark_clean")


In [0]:
display(df_clean.tail(10))
df_clean.printSchema()

State,District,Market,Commodity,Variety,Grade,Arrival_Date,Min_Price,Max_Price,Modal_Price,Commodity_Code
Uttar Pradesh,Khiri (Lakhimpur),Maigalganj,Cabbage,Cabbage,FAQ,2025-11-06,1800.0,1860.0,1830.0,154
Uttar Pradesh,Khiri (Lakhimpur),Maigalganj,Gur (Jaggery),Pathari,FAQ,2025-11-06,3620.0,3660.0,3640.0,74
Uttar Pradesh,Khiri (Lakhimpur),Maigalganj,Apple,Kasmir/Shimla - II,Medium,2025-11-06,9800.0,9860.0,9830.0,17
Uttar Pradesh,Ghazipur,Jamanian,Green Chilli,Green Chilly,FAQ,2025-11-06,1500.0,1600.0,1550.0,87
Uttar Pradesh,Khiri (Lakhimpur),Maigalganj,Pumpkin,Pumpkin,FAQ,2025-11-06,2400.0,2449.0,2430.0,84
Gujarat,Bharuch,Bharuch,Banana - Green,Other,FAQ,2025-11-06,400.0,700.0,500.0,90
Gujarat,Bharuch,Bharuch,Bhindi (Ladies Finger),Other,FAQ,2025-11-06,2000.0,2700.0,2200.0,85
Gujarat,Bharuch,Bharuch,Bottle gourd,Other,FAQ,2025-11-06,1000.0,2000.0,1500.0,82
Gujarat,Bharuch,Bharuch,Onion,Other,Non-FAQ,2025-11-06,1000.0,1700.0,1300.0,23
Uttar Pradesh,Khiri (Lakhimpur),Maigalganj,Wheat,Dara,FAQ,2025-11-06,2400.0,2465.0,2430.0,1


root
 |-- State: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Market: string (nullable = true)
 |-- Commodity: string (nullable = true)
 |-- Variety: string (nullable = true)
 |-- Grade: string (nullable = true)
 |-- Arrival_Date: string (nullable = true)
 |-- Min_Price: double (nullable = true)
 |-- Max_Price: double (nullable = true)
 |-- Modal_Price: double (nullable = true)
 |-- Commodity_Code: long (nullable = true)



# Data Preprocessing

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# ===============================
# State → Region lookup table
# ===============================
region_lookup = spark.createDataFrame([
    ("Punjab", "North"),
    ("Haryana", "North"),
    ("Uttar Pradesh", "North"),
    ("Delhi", "North"),
    ("Himachal Pradesh", "North"),
    ("Jammu and Kashmir", "North"),
    ("Maharashtra", "West"),
    ("Gujarat", "West"),
    ("Rajasthan", "West"),
    ("Goa", "West"),
    ("Karnataka", "South"),
    ("Tamil Nadu", "South"),
    ("Kerala", "South"),
    ("Andhra Pradesh", "South"),
    ("Telangana", "South"),
    ("Bihar", "East"),
    ("Odisha", "East"),
    ("Jharkhand", "East"),
    ("West Bengal", "East"),
    ("Assam", "North-East"),
    ("Nagaland", "North-East"),
    ("Manipur", "North-East"),
    ("Tripura", "North-East"),
    ("Sikkim", "North-East"),
], ["State", "Region"])


# ===============================
# 2. Main preprocessing pipeline
# ===============================
df_preprocessed = (
    df_clean

    # ----- FILTERS -----
    .filter(F.col("Arrival_Date") >= "2016-01-01")
    .filter(
        (F.col("Min_Price").isNotNull()) &
        (F.col("Max_Price").isNotNull()) &
        (F.col("Modal_Price").isNotNull()) &
        (F.col("Max_Price") >= F.col("Min_Price"))
    )

    # ----- COLUMN TRANSFORMATIONS -----
    .withColumn("Arrival_Date", F.to_date("Arrival_Date", "yyyy-MM-dd"))
    .withColumn("Year", F.year("Arrival_Date"))
    .withColumn("Month", F.month("Arrival_Date"))
    .withColumn("Quarter", F.quarter("Arrival_Date"))
    .withColumn("Price_Spread", F.col("Max_Price") - F.col("Min_Price"))
    .withColumn("Modal_Price_Kg", F.col("Modal_Price") / 100)
    .withColumn("Max_Min_Ratio", F.try_divide(F.col("Max_Price"), F.col("Min_Price")))
    .withColumn("State_Upper", F.upper("State"))
    .withColumn("District_Upper", F.upper("District"))

    # ----- STATE → REGION JOIN -----
    .join(F.broadcast(region_lookup), "State", "left")
)


# A) Market-level daily summary

In [0]:
df_market_agg = (
    df_preprocessed
        .groupBy("State", "Region", "District", "Market", "Arrival_Date")
        .agg(
            F.count("*").alias("Num_Records"),
            F.avg("Modal_Price").alias("Avg_Modal_Price"),
            F.avg("Price_Spread").alias("Avg_Spread"),
            F.max("Max_Price").alias("Daily_Max_Price")
        )
)
display(df_market_agg.head(5))

State,Region,District,Market,Arrival_Date,Num_Records,Avg_Modal_Price,Avg_Spread,Daily_Max_Price
Andhra Pradesh,South,Anantapur,Anantapur,2016-01-01,2,5450.0,550.0,6900.0
Gujarat,West,Dahod,Dahod,2016-01-01,16,4105.0,209.875,10000.0
Karnataka,South,Bellary,Hoovinahadagali,2016-01-01,1,1380.0,0.0,1380.0
Odisha,East,Bhadrak,Bhadrak,2016-01-01,7,1672.857142857143,690.0,5000.0
Andhra Pradesh,South,Vijayanagaram,Gajapathinagaram,2016-01-02,4,5127.5,577.5,13000.0


#B) Commodity rolling 30-day window

In [0]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

window_spec = (
    Window
        .partitionBy("Commodity")
        .orderBy(F.to_date("Arrival_Date"))
        .rowsBetween(-7, 0)
)

df_rolling = (
    df_preprocessed
        .withColumn("Volatility_7D", F.stddev("Modal_Price").over(window_spec))
        .withColumn("Spread_7D_Avg", F.avg("Price_Spread").over(window_spec))
)

df_final = df_rolling
display(df_final.limit(5))


State,District,Market,Commodity,Variety,Grade,Arrival_Date,Min_Price,Max_Price,Modal_Price,Commodity_Code,Year,Month,Quarter,Price_Spread,Modal_Price_Kg,Max_Min_Ratio,State_Upper,District_Upper,Region,Arrival_Date_Date,Volatility_7D,Spread_7D_Avg
Madhya Pradesh,Neemuch,Neemuch,Absinthe,chirayta,FAQ,2024-08-20,2125.0,2125.0,2125.0,451,2024,8,3,0.0,21.25,1.0,MADHYA PRADESH,NEEMUCH,null,2024-08-20,null,0.0
Madhya Pradesh,Neemuch,Neemuch,Absinthe,chirayta,FAQ,2024-08-22,1500.0,1500.0,1500.0,451,2024,8,3,0.0,15.0,1.0,MADHYA PRADESH,NEEMUCH,null,2024-08-22,441.9417382415922,0.0
Madhya Pradesh,Umariya,Umariya,Absinthe,chirayta,FAQ,2024-09-01,2400.0,2400.0,2400.0,451,2024,9,3,0.0,24.0,1.0,MADHYA PRADESH,UMARIYA,null,2024-09-01,461.203136734057,0.0
Madhya Pradesh,Neemuch,Neemuch,Absinthe,chirayta,FAQ,2024-09-04,1850.0,1911.0,1911.0,451,2024,9,3,61.0,19.11,1.032972972972973,MADHYA PRADESH,NEEMUCH,null,2024-09-04,379.7025151352042,15.25
Madhya Pradesh,Khandwa,Khandwa,Absinthe,chirayta,FAQ,2024-09-06,1540.0,1545.0,1545.0,451,2024,9,3,5.0,15.45,1.0032467532467533,MADHYA PRADESH,KHANDWA,null,2024-09-06,382.9813311376939,13.2


In [0]:
%sql
DROP VOLUME IF EXISTS workspace.default.preprocessed;
CREATE VOLUME workspace.default.preprocessed;

In [0]:
df_final.write.mode("overwrite").parquet("/Volumes/workspace/default/preprocessed")

In [0]:
%sql
DROP TABLE IF EXISTS workspace.default.preprocessed_data;

In [0]:
df_final.write.mode("overwrite").saveAsTable("workspace.default.preprocessed_data")

# Queries

#1) Top 3 Cheapest Markets per Commodity ( Monthly Rolling Trend )

In [0]:
query1 = spark.sql('''
WITH rolling AS (
    SELECT
        Commodity,
        Market,
        date_format(Arrival_Date, 'yyyy-MM') AS YearMonth,
        Modal_Price,
        Price_Spread,
        AVG(Modal_Price) OVER (
            PARTITION BY Commodity
            ORDER BY Arrival_Date
            ROWS BETWEEN 14 PRECEDING AND CURRENT ROW
        ) AS Rolling14
    FROM preprocessed_data
    WHERE Arrival_Date >= '2018-01-01'
),

monthly_agg AS (
    SELECT
        Commodity,
        YearMonth,
        Market,
        AVG(Rolling14) AS Avg_Roll14,
        AVG(Price_Spread) AS Avg_Spread,
        COUNT(*) AS Records
    FROM rolling
    GROUP BY Commodity, YearMonth, Market
),

ranked AS (
    SELECT
        *,
        RANK() OVER (
            PARTITION BY Commodity, YearMonth
            ORDER BY Avg_Roll14 ASC
        ) AS Rank
    FROM monthly_agg
)

SELECT *
FROM ranked
WHERE Rank <= 3
ORDER BY Commodity, YearMonth, Rank;
''')
display(query1.head(5))


Commodity,YearMonth,Market,Avg_Roll14,Avg_Spread,Records,Rank
Absinthe,2024-08,Neemuch,1968.75,0.0,2,1
Absinthe,2024-09,Khandwa,1896.2,5.0,1,1
Absinthe,2024-09,Neemuch,2046.5968614718613,15.25,4,2
Absinthe,2024-09,Umariya,2081.4263888888886,0.0,4,3
Absinthe,2024-10,Umariya,2204.8416666666667,0.0,2,1


In [0]:
query2 = spark.sql('''
WITH stats AS (
    SELECT
        State,
        Market,
        Arrival_Date,
        date_format(Arrival_Date, 'yyyy-MM') AS YearMonth,
        Modal_Price,

        -- 60-day moving average
        AVG(Modal_Price) OVER (
            PARTITION BY Market
            ORDER BY Arrival_Date
            ROWS BETWEEN 60 PRECEDING AND CURRENT ROW
        ) AS MA60,

        -- 60-day moving standard deviation
        STDDEV(Modal_Price) OVER (
            PARTITION BY Market
            ORDER BY Arrival_Date
            ROWS BETWEEN 60 PRECEDING AND CURRENT ROW
        ) AS STD60
    FROM preprocessed_data
),

zscores AS (
    SELECT
        *,
        TRY_DIVIDE(Modal_Price - MA60, STD60) AS ZScore60
    FROM stats
)

SELECT
    State,
    Market,
    YearMonth,
    AVG(ZScore60) AS Avg_ZScore,
    AVG(STD60) AS Avg_STD60,
    MAX(STD60) AS Peak_STD60,
    COUNT(*) AS Num_Records
FROM zscores
GROUP BY State, Market, YearMonth
ORDER BY Market, YearMonth;
'''
)

display(query2.head(5))

State,Market,YearMonth,Avg_ZScore,Avg_STD60,Peak_STD60,Num_Records
Madhya Pradesh,A lot,2016-01,0.15123273356870529,1125.6733808338179,1353.402379191052,66
Madhya Pradesh,A lot,2016-02,0.2158842913290821,1319.1653378185151,1430.9239729606331,107
Madhya Pradesh,A lot,2016-03,-0.06182616257940493,1303.770768943858,1369.020011070102,107
Madhya Pradesh,A lot,2016-04,0.05906139891877386,1427.183243343393,1593.878671737149,93
Madhya Pradesh,A lot,2016-05,-0.06567114019024524,1580.3750846500368,1644.2729648875547,88


# Optimsed V/S Unoptimised Queries

### Example 1: Yearly Average Modal Price per Commodity

In [0]:
# Unoptimised
from pyspark.sql import functions as F

df_bad = (
    df_clean
        .withColumn("Year", F.year("Arrival_Date"))
        .repartition(2000)                                 # unnecessary massive shuffle
        .groupBy("Commodity", "Year")                      # groups ALL years
        .agg(F.avg("Modal_Price").alias("AvgPrice"))
        .filter("Year >= 2024")                            # filtering too late
)

df_bad_result = df_bad.orderBy("Commodity", "Year")
df_bad_result.count()


642

In [0]:
# Optimised
df_opt = (
    df_clean
        .withColumn("Year", F.year("Arrival_Date"))
        .filter("Year >= 2024")                            # early filter
        .select("Commodity", "Year", "Modal_Price")        # prune unnecessary columns
        .repartition("Year")                               # cluster same Year values
        .groupBy("Commodity", "Year")
        .agg(F.avg("Modal_Price").alias("AvgPrice"))
)

df_opt_result = df_opt.orderBy("Commodity", "Year")
df_opt_result.count()


642

### Explanation
This optimised query filters the data early and prunes unused columns, so Spark only scans and processes the rows and fields it actually needs.
By repartitioning on Year, it avoids a huge shuffle during the groupBy, leading to a noticeably faster execution — roughly a 2.5X - 3X speedup over the unoptimised version.

![](/Workspace/Users/kedarvaidya0504@gmail.com/ex1_un.png)

## V/S

![](/Workspace/Users/kedarvaidya0504@gmail.com/ex1_opt.png)

In [0]:
df_opt_result.write.mode("overwrite").format("delta").saveAsTable("default.Yearly")

### Example 2: 30-Day Rolling Average Price per Commodity

In [0]:
# Unoptimised

from pyspark.sql import functions as F
from pyspark.sql.window import Window

# A lookup table of commodity codes (small)
lookup = df_clean.select("Commodity_Code").distinct()

df_bad_complex = (
    df_clean
        .join(lookup, "Commodity_Code")                        # huge join early
        .withColumn("Year", F.year("Arrival_Date"))             # compute Year early
        .repartition(2500)                                      # random massive shuffle
        .withColumn(
            "RollingAvg30",
            F.avg("Modal_Price").over(
                Window.partitionBy("Commodity")
                      .orderBy("Arrival_Date")
                      .rowsBetween(-30, 0)
            )
        )                                                       # window on full data
        .filter("Year >= 2024")                                 # filtering too late
        .select("Commodity", "Market", "Arrival_Date", "RollingAvg30")
)

df_bad_complex_result = df_bad_complex.orderBy("Commodity", "Arrival_Date")
display(df_bad_complex_result.head(5))

Commodity,Market,Arrival_Date,RollingAvg30
Absinthe,Neemuch,2024-08-20,2125.0
Absinthe,Neemuch,2024-08-22,1812.5
Absinthe,Umariya,2024-09-01,2008.3333333333333
Absinthe,Neemuch,2024-09-04,1984.0
Absinthe,Khandwa,2024-09-06,1896.2


In [0]:
# Optimised

from pyspark.sql import functions as F
from pyspark.sql.window import Window

lookup = df_clean.select("Commodity_Code").distinct()

df_opt_complex = (
    df_clean
        .withColumn("Year", F.year("Arrival_Date"))
        .filter("Year >= 2024")                               # early filter
        .select("Commodity_Code", "Commodity", "Market", "Arrival_Date", "Modal_Price")
        .join(F.broadcast(lookup), "Commodity_Code")          # no shuffle
        .repartition("Commodity")                             # co-locate rows for window agg
        .withColumn(
            "RollingAvg30",
            F.avg("Modal_Price").over(
                Window.partitionBy("Commodity")
                      .orderBy("Arrival_Date")
                      .rowsBetween(-30, 0)
            )
        )
)

df_opt_complex_result = df_opt_complex.orderBy("Commodity", "Arrival_Date")
display(df_opt_complex_result.head(5))

Commodity_Code,Commodity,Market,Arrival_Date,Modal_Price,RollingAvg30
451,Absinthe,Neemuch,2024-08-20,2125.0,2125.0
451,Absinthe,Neemuch,2024-08-22,1500.0,1812.5
451,Absinthe,Umariya,2024-09-01,2400.0,2008.3333333333333
451,Absinthe,Neemuch,2024-09-04,1911.0,1984.0
451,Absinthe,Khandwa,2024-09-06,1545.0,1896.2



### Explanation
The optimised version applies filters and column pruning early and avoids unnecessary shuffles, so Spark processes far less data before doing expensive operations like joins and window functions.
Because of that, the query runs much faster end-to-end, giving roughly a 2–2.5× speedup compared to the unoptimised version.

![](/Workspace/Users/kedarvaidya0504@gmail.com/ex2_un)

## V/S

![](/Workspace/Users/kedarvaidya0504@gmail.com/ex_opt)




In [0]:
%sql
DROP VOLUME IF EXISTS workspace.default.RollingAvg30;
CREATE VOLUME workspace.default.RollingAvg30;

In [0]:
df_opt_complex_result.write.mode("overwrite").parquet("/Volumes/workspace/default/RollingAvg30")


# Performance Analysis

In [0]:
print("UNOPTIMIZED PLAN:")
df_bad.explain("formatted")

print("UNOPTIMIZED COUNT:")
bad_count = df_bad.count()

UNOPTIMIZED PLAN:
== Physical Plan ==
AdaptiveSparkPlan (63)
+- == Initial Plan ==
   ColumnarToRow (62)
   +- PhotonResultStage (61)
      +- PhotonGroupingAgg (60)
         +- PhotonShuffleExchangeSource (59)
            +- PhotonShuffleMapStage (58)
               +- PhotonShuffleExchangeSink (57)
                  +- PhotonGroupingAgg (56)
                     +- PhotonShuffleExchangeSource (55)
                        +- PhotonShuffleMapStage (54)
                           +- PhotonShuffleExchangeSink (53)
                              +- PhotonSort (52)
                                 +- PhotonUnion (51)
                                    :- PhotonProject (2)
                                    :  +- PhotonScan parquet  (1)
                                    :- PhotonProject (4)
                                    :  +- PhotonScan parquet  (3)
                                    :- PhotonProject (6)
                                    :  +- PhotonScan parquet  (5)
           

In [0]:
print("OPTIMIZED PLAN:")
df_opt.explain("formatted")

print("OPTIMIZED COUNT:")
opt_count = df_opt.count()

OPTIMIZED PLAN:
== Physical Plan ==
AdaptiveSparkPlan (58)
+- == Initial Plan ==
   ColumnarToRow (57)
   +- PhotonResultStage (56)
      +- PhotonGroupingAgg (55)
         +- PhotonShuffleExchangeSource (54)
            +- PhotonShuffleMapStage (53)
               +- PhotonShuffleExchangeSink (52)
                  +- PhotonUnion (51)
                     :- PhotonProject (2)
                     :  +- PhotonScan parquet  (1)
                     :- PhotonProject (4)
                     :  +- PhotonScan parquet  (3)
                     :- PhotonProject (6)
                     :  +- PhotonScan parquet  (5)
                     :- PhotonProject (8)
                     :  +- PhotonScan parquet  (7)
                     :- PhotonProject (10)
                     :  +- PhotonScan parquet  (9)
                     :- PhotonProject (12)
                     :  +- PhotonScan parquet  (11)
                     :- PhotonProject (14)
                     :  +- PhotonScan parquet  (13)
      

## Explanation

In the unoptimised version of the query, Spark ends up doing a lot of unnecessary work. The code forces Spark to scan every parquet file, shuffle a huge amount of data, and only later apply the filter for Year >= 2024. Because the filter comes so late, Spark first processes millions of rows and then throws most of them away. The plan also shows a repartition(2000), which creates a massive shuffle across the cluster for no good reason. This leads to lots of expensive stages like PhotonShuffleExchange and PhotonShuffleMapStage, which slow everything down.

Even though the code is not written well, Spark still tries to be smart. It automatically pushes the year filter down into the file scan step, which means it skips reading row groups that don’t match. Spark also does “column pruning”, meaning it only loads the columns that are actually used in the query. These built-in optimisations help, but they can’t fix the huge shuffle caused by the bad repartition and late filtering.

The optimised query is much cleaner and way faster. It applies the filter before any heavy operations, so Spark only processes the data it needs. It also selects only the required columns early on, reducing unnecessary I/O. Instead of randomly doing repartition(2000), the optimised query uses repartition("Year"), which groups data by the same year and avoids a giant shuffle. Because of these changes, the physical plan becomes much smaller, with fewer shuffle stages and no extra sorting. Overall, the optimised version runs faster, uses fewer resources, and takes advantage of Spark’s built-in optimisations properly.

# 3. Actions vs Transformations

In [0]:
df = spark.read.parquet("/Volumes/workspace/default/pyspark_clean")
print("Dataset loaded.")

Dataset loaded.


In [0]:
from pyspark.sql import functions as F

# Transformation 1: filter rows after 2020
df_filtered = df.filter(F.col("Arrival_Date") >= "2020-01-01")

# Transformation 2: add Price Spread column
df_with_spread = df_filtered.withColumn(
    "Price_Spread", F.col("Max_Price") - F.col("Min_Price")
)

# Transformation 3: extract year from date
df_final = df_with_spread.withColumn("Year", F.year("Arrival_Date"))

print("Transformations defined. No Spark job has run yet.")


Transformations defined. No Spark job has run yet.


In [0]:
print("Triggering first action with show():")
display(df_final.show(5))


Triggering first action with show():
+--------------+------------+--------------------+--------------+-------------+-----+------------+---------+---------+-----------+--------------+------------+----+
|         State|    District|              Market|     Commodity|      Variety|Grade|Arrival_Date|Min_Price|Max_Price|Modal_Price|Commodity_Code|Price_Spread|Year|
+--------------+------------+--------------------+--------------+-------------+-----+------------+---------+---------+-----------+--------------+------------+----+
|Andhra Pradesh|     Chittor|            Chittoor| Gur (Jaggery)|         NO 2|  FAQ|  2025-01-01|   3200.0|   3500.0|     3500.0|            74|       300.0|2025|
|    Tamil Nadu|The Nilgiris|Gudalur (Uzhavar ...|        Garlic|      Average|Local|  2025-01-01|  28000.0|  30000.0|    30000.0|            25|      2000.0|2025|
|    Tamil Nadu|The Nilgiris|Gudalur (Uzhavar ...|Ginger (Green)| Green Ginger|Local|  2025-01-01|   7000.0|   8000.0|     8000.0|           10

In [0]:
print("Triggering another action (count()):")
df_final.count()

Triggering another action (count()):


27466611

# ML-LIB

In [0]:
df = spark.read.parquet("/Volumes/workspace/default/pyspark_clean")

In [0]:
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# Feature engineering (if not already done)
df_ml = (
    df
    .withColumn("Price_Spread", F.col("Max_Price") - F.col("Min_Price"))
    .select(
        "Modal_Price",
        "Min_Price",
        "Max_Price",
        "Price_Spread",
        "Commodity_Code"
    )
    .dropna()
)


In [0]:
assembler = VectorAssembler(
    inputCols=["Min_Price", "Max_Price", "Price_Spread", "Commodity_Code"],
    outputCol="features"
)

df_features = assembler.transform(df_ml)


In [0]:
train_df, test_df = df_features.randomSplit([0.8, 0.2], seed=42)


In [0]:
lr = LinearRegression(
    featuresCol="features",
    labelCol="Modal_Price"
)

lr_model = lr.fit(train_df)


In [0]:
predictions = lr_model.transform(test_df)
predictions.select("Modal_Price", "prediction").show(10)


+-----------+-----------------+
|Modal_Price|       prediction|
+-----------+-----------------+
|        1.0|368.5935170255327|
|        1.0|419.2263044773363|
|        1.0|419.2263044773363|
|        1.0|419.2263044773363|
|        1.0|419.2263044773363|
|        1.0|419.2263044773363|
|        1.0|419.2263044773363|
|        1.0|419.2263044773363|
|        1.0|419.2263044773363|
|        1.0|419.2263044773363|
+-----------+-----------------+
only showing top 10 rows


In [0]:
print("RMSE:", lr_model.summary.rootMeanSquaredError)
print("R2:", lr_model.summary.r2)


RMSE: 1567.4901111068248
R2: 0.9999142741087812
